In [1]:
model_name = 'unsloth/llama-3-8b-Instruct-bnb-4bit'
model_alias = '0-shot-8b-instruct'

In [2]:
from unsloth import FastLanguageModel
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
import csv
import re

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
max_seq_length = 20 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

FastLanguageModel.for_inference(model)

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: NVIDIA A30 MIG 2g.12gb. Max memory: 11.688 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [359]:
test_dataset = load_dataset(
    "msc-smart-contract-audition/audits-with-reasons",
    split="test"
)

test_dataset

Dataset({
    features: ['code', 'description', 'recommendation', 'type', 'functionality'],
    num_rows: 437
})

In [5]:
query_template = \
"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Below are one or more Solidity codeblocks. The codeblocks might contain vulnerable code.
If there is a vulnerability please provide a description of the vulnearblity in terms of the code that is responsible for it.
Describe how an attacker would be able to take advantage of the vulnerability so the explanation is even more clear.

If there is no vulnerability output "There is no vulnearbility".
Do not output any ways to mitigate or fix the vulnerability, only the vulnerability itself.

Think step by step about the code and the possible vulnerabilities.
<|start_header_id|>user<|end_header_id|>
Codeblocks:
{}<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
Description of the vulnerability:
"""

In [39]:
!pwd

/homes/kza23


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [360]:
df_test = test_dataset.to_pandas()

queries = df_test.apply(lambda row: query_template.format(row['code'].replace('\\n', '\n')), axis=1)

print(queries.iloc[0])

NameError: name 'query_template' is not defined

In [9]:
with open(f"{model_alias}-descriptions.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["output", "real"])

    for query, real in tqdm(zip(queries, test_dataset['description']), total=len(queries)):
        inputs = tokenizer(query, return_tensors="pt", truncation=True).to("cuda")
        output_tokens = model.generate(
            **inputs, max_new_tokens=512, pad_token_id=tokenizer.pad_token_id
        )
        decoded_output = tokenizer.decode(
            output_tokens[0],
            skip_special_tokens=True,
            pad_token_id=tokenizer.pad_token_id,
        )
        description = decoded_output.split("Description of the vulnerability:\n")[1].strip().replace("\n", "\\n")
        writer.writerow([description, real])

100%|████████████████████████████████████████████████████████████| 437/437 [1:00:39<00:00,  8.33s/it]


In [365]:
results = pd.read_csv(f"{model_alias}-descriptions.csv")
results.loc[results["real"].isnull(), 'real'] = "There is no vulnearbility"

results

,output,real
0,The vulnerability is in the `netPrimeDebtChang...,The `repayAccountPrimeDebtAtSettlement()` func...
1,The vulnerability is in the `claimableUnlocked...,The `lockCapital` mechanism in the ProtectionP...
2,The vulnerability is in the `_update` function...,The vulnerability occurs when the `_invariant`...
3,The codeblock contains a vulnerability in the ...,There is no vulnearbility
4,The vulnerability is in the `_credit` function...,The vulnerability arises when the account bein...
...,...,...
432,The vulnerability is in the `require` statemen...,There is no vulnearbility
433,The vulnerability is in the `getPriceUSD` func...,The StableOracleWBTC contract relies on a BTC/...
434,The vulnerability is in the `MigrateWithdrawal...,The `MigrateWithdrawal` function in the `migra...
435,The vulnerability is in the `addLiquidity` fun...,The createMarket transaction lacks a crucial e...


In [ ]:
comparison_query_template = \
"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Below is text comprehension task. Output a score based on the specification below.

<|start_header_id|>user<|end_header_id|>

Below are two audits for a piece of solidity code.
You are a security auditor and your task is to mark an audit written by a student based on 3 critera.

First reason about whether each criterion is met or not.
If criteria are not fully satisifed, determine which is the more appropriate label from "PASS" or "FAIL" (nothing else is allowed).
Only after you have explained the reasoning behind each criterion, output a list labeling each criterion with either "PASS" or "FAIL".

Reason about wether the student's description satisfies the following criteria:
Criterion 1: Fulfilled if the two audits describe an identical vulnerability. This is about the nature of vulnerability and how it affects the contract, rather than the specifics of the code.
Criterion 2: Fulfilled if the student describes the same code scope or a subset of yours. In other words partial scope is also acceptable. Otherwise, this criterion is not met.
Criterion 3: Fulfilled if the two audits describe identical attacking strategy. The description does NOT need to be identical just the idea for the attack.

Only once you have deliberated on the criteria, output EXACTLY 3 criteria with a status of either "PASS" or "FAIL".
Example: Criterion 1: PASS

Do not output anything else like clarifications or other remarks after the list of criteria.

Your audit:
{}<|eot_id|>

Student's audit:
{}<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
My reasoning about the criteria:
"""

: 

In [364]:
def run_query(data, query, show_tqdm=True):
    queries = data.apply(lambda row: query.format(
        row['real'].replace('\\n', '\n'),
        row['output'].replace('\\n', '\n'),
    ), axis=1)

    if show_tqdm:
        iterator = tqdm(enumerate(queries), total=len(queries))
    else:
        iterator = enumerate(queries)

    for idx, query in iterator:
        real_contains_vuln = data.iloc[idx]["real"] != "There is no vulnearbility"
        output_contains_vuln = data.iloc[idx]["output"] != "There is no vulnearbility"

        if real_contains_vuln != output_contains_vuln:
            yield ["FAIL", "FAIL", "FAIL", "FAIL", "Criterion 0: FAIL\\n One of the descriptions does not contain a vulnerability."]
            continue

        if not real_contains_vuln and not output_contains_vuln:
            yield ["PASS", "PASS", "PASS", "PASS", "No vulnerabilities to compare."]
            continue

        inputs = tokenizer(query, return_tensors="pt", truncation=True).to("cuda")
        output_tokens = model.generate(
            **inputs, max_new_tokens=512, pad_token_id=tokenizer.pad_token_id
        )
        decoded_output = tokenizer.decode(
            output_tokens[0],
            skip_special_tokens=True,
            pad_token_id=tokenizer.pad_token_id,
        )

        result = decoded_output.split("My reasoning about the criteria:\n")[1].strip()
        pattern = re.compile(r'Criterion \d+: (PASS|FAIL)')
        criteria = ["PASS"] + pattern.findall(result)[-3:] # Add a pass for the 0th criterion

        yield criteria + [result.replace("\n", "\\n")]

In [367]:
with open(f"{model_alias}-descriptions-results-v2.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["cr0", "cr1", "cr2", "cr3", "description"])
    for result in run_query(results, comparison_query_template):
        writer.writerow(result)

100%|████████████████████████████████████████████████████████| 437/437 [46:15<00:00,  6.35s/it]


## ANALYSIS

# Correct entries:

197, 240, 314: cr1 should FAIL, but everything else should be PASS

284, 389: PASS fully

# Entries to look into:


## 316: Should pass everything!!! - (Fix all criteria) 
#### Criterion 1: FAIL
The junior auditor's description mentions an integer overflow attack, while the ground truth mentions an integer overflow vulnerability in the calculation of `tokenAmtForAmm`. Although the consequences are similar, the vulnerability types are different, so this criterion is not met.

#### Criterion 2: FAIL
The junior auditor's description does not specify the exact part of the code where the vulnerability is located, while the ground truth explicitly mentions the `_tokenAmountsForLiquidityBootstrap` function. Since the location of the vulnerability is not the same, this criterion is not met.

#### Criterion 3: FAIL
The junior auditor's description mentions that an attacker can manipulate the values of `virtualEth`, `initialTokenMatch`, `bootstrapEth`, and `totalEth` to cause an integer overflow, while the ground truth mentions that an attacker can set these variables to their maximum possible values to achieve the overflow. Although the methods of exploitation are similar, the exact steps are different, so this criterion is not met.

## 392: Should PASS everything!!! - (Fix all criteria)
#### Criterion 1: FAIL
The junior auditor's description does not mention the same type of vulnerability as the ground truth. The ground truth describes a vulnerability in the `view` and `virtual` functions, while the junior auditor's description mentions a vulnerability in the `view` and `virtual` functions, but does not specify the type of vulnerability.

#### Criterion 2: FAIL
The junior auditor's description does not mention the same part of the code as the ground truth. The ground truth describes a vulnerability in the `StrategyBase` contract, specifically in the `underlyingToShares` and `sharesToUnderlying` functions, while the junior auditor's description does not specify the location of the vulnerability.

#### Criterion 3: PASS
The junior auditor's description describes the same way an attacker would take advantage of the vulnerability as the ground truth. Both descriptions mention the creation of a child contract that overrides the `underlyingToShares` and `sharesToUnderlying` functions and the use of the overridden functions to manipulate the calculations."

## 379: Should PASS cr1 - vulnerability is not mentioned explicitly, but the description matches exactly
#### Criterion 1: FAIL
The junior auditor's description does not mention the vulnerability type explicitly, whereas the ground truth does. The junior auditor's description only provides an explanation of the vulnerability, whereas the ground truth explicitly states that it is related to the `onlyEOAEx` modifier and the introduction of EIP 3074.

#### Criterion 2: PASS
Both audits describe the vulnerability in the same part of the code, specifically the `onlyEOAEx` modifier.

#### Criterion 3: PASS
Both audits describe the same way an attacker would take advantage of the vulnerability, which is by creating a contract that relays the call to the current contract."

## 135: The model outputs N/A which shouldnt’t be a valid option
#### Criterion 1: FAIL
The two audits describe different types of vulnerabilities. The ground truth describes a vulnerability related to the Chainlink aggregator returning an incorrect price, while the auditor's description describes a denial-of-service (DoS) attack.

#### Criterion 2: N/A
Since the two audits describe different types of vulnerabilities, this criterion is not applicable.

#### Criterion 3: N/A
Since the two audits describe different types of vulnerabilities, this criterion is not applicable


## 32: The location should be correct as it regards the same function
#### Criterion 1: FAIL
The junior auditor's description mentions an integer overflow attack, while the ground truth describes an underflow vulnerability. The types of vulnerabilities are different, so this criterion is not met.

#### Criterion 2: FAIL
The junior auditor's description does not specify the exact part of the code where the vulnerability is located, while the ground truth explicitly mentions the calculation of `borrowingCollateral` as the vulnerable part. The locations of the vulnerabilities are different, so this criterion is not met.

#### Criterion 3: FAIL
The junior auditor's description describes an attacker manipulating the values of `cache.borrowedAmount` and `cache.holdTokenBalance` to cause an integer overflow, while the ground truth describes an attacker causing an underflow by exceeding `cache.holdTokenBalance`. The methods of exploitation differ, so this criterion is not met.

In [352]:
correct_output = pd.DataFrame([
    [197, 'PASS', 'PASS', 'PASS', 'PASS'],
    [240, 'PASS', 'FAIL', 'PASS', 'FAIL'],
    [314, 'PASS', 'FAIL', 'FAIL', 'FAIL'],
    [284, 'PASS', 'PASS', 'PASS', 'PASS'],
    [389, 'PASS', 'PASS', 'PASS', 'PASS'],
    [57,  'PASS', 'PASS', 'PASS', 'PASS'], # The codeblocks actually contain example code demonstrating the vulnerability so this one is not a fair comparison
    [316, 'PASS', 'PASS', 'PASS', 'PASS'],
    [392, 'PASS', 'PASS', 'FAIL', 'PASS'],
    [379, 'PASS', 'PASS', 'PASS', 'PASS'],
    [135, 'PASS', 'FAIL', 'FAIL', 'FAIL'],
    [32,  'PASS', 'FAIL', 'PASS', 'FAIL'],
    [2,   'PASS', 'FAIL', 'PASS', 'FAIL'],
    [46,  'PASS', 'FAIL', 'FAIL', 'FAIL'],
], columns=["index", "cr0", "cr1", "cr2", "cr3"]).set_index("index")

In [356]:
correct_output

,cr0,cr1,cr2,cr3
index,,,,
197,PASS,PASS,PASS,PASS
240,PASS,FAIL,PASS,FAIL
314,PASS,FAIL,FAIL,FAIL
284,PASS,PASS,PASS,PASS
389,PASS,PASS,PASS,PASS
57,PASS,PASS,PASS,PASS
316,PASS,PASS,PASS,PASS
392,PASS,PASS,FAIL,PASS
379,PASS,PASS,PASS,PASS


In [180]:
data = pd.read_csv(f"{model_alias}-descriptions.csv")

In [282]:
check_idx = 240

print(data.iloc[check_idx]['real'].replace('\\n', '\n'))
print()
print("=" * 200)
print()
print(data.iloc[check_idx]['output'].replace('\\n', '\n'))

The protocol's `Swap` library contains a hardcoded address for WETH (Wrapped Ether), which is a token on the Ethereum blockchain. This hardcoded address is set to `0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2`. While the protocol's README.md mentions support for multiple EVM-based blockchains, including Ethereum Mainnet, Arbitrum, Optimism, Polygon, and Binance Smart Chain, the hardcoded WETH address in the `Swap` library may cause issues on these chains. Specifically, on chains like Polygon, which is not the Ethereum Mainnet, the protocol's functionality may not work as intended due to the hardcoded WETH address. This is because the address is not compatible with the WETH contract on these alternative chains.


The codeblock contains a hardcoded address of a contract, specifically the WETH (Wrapped Ether) contract. This is not a vulnerability in itself, but it could be a potential vulnerability if the address is not properly validated.

Description of how an attacker could take advantag

In [357]:
test_data = data.join(correct_output, how="inner")
test_data = test_data.join(test_dataset.to_pandas()["code"], how="inner")
test_data

,output,real,cr0,cr1,cr2,cr3,code
2,The vulnerability is in the `_update` function...,The vulnerability occurs when the `_invariant`...,PASS,FAIL,PASS,FAIL,```\nfunction _invariant(\n Context memory ...
32,The codeblock is vulnerable to an integer over...,The vulnerability arises from an incorrect cal...,PASS,FAIL,PASS,FAIL,```\nuint256 borrowingCollateral = cache.borro...
46,The code is vulnerable to a reentrancy attack....,The `setValidatorAddress` function in the smar...,PASS,FAIL,FAIL,FAIL,```\nfunction setValidatorAddress(uint128 vali...
57,The vulnerability is in the `clearRequest` fun...,The `CoolerCallback.isCoolerCallback()` functi...,PASS,PASS,PASS,PASS,```\nfunction clearRequest(\n uint256 reqID...
135,The code is vulnerable to a denial-of-service ...,The D3Oracle vulnerability occurs when the Cha...,PASS,FAIL,FAIL,FAIL,"```\n(uint80 roundID, int256 price, uint256 up..."
197,The code is vulnerable to a logical error.\n\n...,The `processAuction()` function in the `VaultA...,PASS,PASS,PASS,PASS,```\nbool cancelled = l.Auction.isCancelled(la...
240,The codeblock contains a hardcoded address of ...,The protocol's `Swap` library contains a hardc...,PASS,FAIL,PASS,FAIL,```\naddress internal constant WETH = 0xC02aaA...
284,The vulnerability is in the `finalise` functio...,The vulnerability in the RocketMinipoolDelegat...,PASS,PASS,PASS,PASS,```\nfinalise() -->\n status == MinipoolStatu...
314,The vulnerability is in the `initiateFlashloan...,"The vulnerability, known as FlasherFTM - Unsol...",PASS,FAIL,FAIL,FAIL,```\n/**\n * @dev Routing Function for Flashlo...
316,The code is vulnerable to an integer overflow ...,The vulnerability allows an attacker to create...,PASS,PASS,PASS,PASS,```\nuint112 virtualEth = type(uint112).max;\n...


In [ ]:
# comparison_query_template_fix_old = \
# """
# <|begin_of_text|><|start_header_id|>system<|end_header_id|>
# Below is text comprehension task. Output a score based on the specification below.

# <|start_header_id|>user<|end_header_id|>

# Below are two descriptions of vulnerabilities for a piece of Solidity code. The solidity code is also provided for context especially for criterion 2.
# You are a security auditor and you need to determine if the two descriptions satisfy several criteria.
# The first audit is written by a junior auditor, and the second one is the ground truth.

# Determine each criterion based on the similarity of the two descriptions.
# The criteria are as follows:

# Criterion 1: Fullfilled if the two audits describe the same type of a vulnerability. For example, pass this criterion if both describe a reentrancy vulnerability or an integer overflow vulnerability.
# Criterion 2: Fullfilled if the auditor describes a vulnerability regarding the same exact part of the code. Do not pass this criterion if the description regards different parts of the code. The scope of the described code in the junior audit shouldn't be too general compared to the ground truth, e.g. just being in the same function fails this criterion, if the ground truth is more specific.
# Criterion 3: Fullfilled if the two audits describe the same way an attacker would take advantage of the vulnerability. Do not pass this criterion if the methods of exploitation differ, even if the outcomes are similar.

# Do not be forgiving, criteria needs to be exactly satisfied.

# First reason about wether each criterion is met or not, without outputting a final labeling of wether the criterion is satisfied or not.
# Only after you have explained the reasoning behind each criterion, output a list labeling each criterion with either "PASS" or "FAIL".

# After your reasoning there should be EXACTLY 3 criteria and their status - either PASS or FAIL.

# Ground truth:
# {}<|eot_id|>

# Auditor's description:
# {}<|eot_id|>

# <|start_header_id|>assistant<|end_header_id|>
# Ground truth details:
# 1. Type of vulnerability:
# """

In [315]:
# comparison_query_template_long = \
# """
# <|begin_of_text|><|start_header_id|>system<|end_header_id|>
# Below is text comprehension task. Output a score based on the specification below.

# <|start_header_id|>user<|end_header_id|>

# Below are two audits for a piece of solidity code. One is your audit which you can assume is always correct.
# You are a security auditor and your task is to mark an audit written by a junior auditor based on 3 critera.

# Before that, determine the following things about your audit:
# 1. Briefly describe what vulnerability have you described. Do NOT pay attention to the vulnerability stated by the junior audtor because it could be wrong.
# 2. What part of the code the vulnerability is in? Provide an exhaustive scope stated in the ground truth specifically.
# 3. How would someone take advantage of the vulnerability according to your audit?

# After you have determined these things, you need to reason about wether the junior auditor's description satisfies the following criteria:
# Criterion 1: Fullfilled if the two audits describe the same vulnerability. Be pedantic about the described vulnearbility.
# Criterion 2: Fullfilled if the junior auditor describes the same scope as you do. The scope of the described code in the junior audit should be the same as yours. It is okay if the junior auditor's scope is a subset of yours. Otherwise, this criterion is not met.
# Criterion 3: Fullfilled if the two audits describe the same attacking method.

# First reason about wether each criterion is met or not, without outputting wether the criterion is satisfied or not.
# Only after you have explained the reasoning behind each criterion, output a list labeling each criterion with either "PASS" or "FAIL".

# There should be EXACTLY 3 criteria.
# First outputing the word "Criterion" followed by the criterion number and then its status separated by a colon and a space.

# Your audit:
# {}<|eot_id|>

# Junior Auditor's audit:
# {}<|eot_id|>

# <|start_header_id|>assistant<|end_header_id|>
# Details about my audit:
# 1. Type of vulnerability:
# """


comparison_query_template_fix = \
"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Below is text comprehension task. Output a score based on the specification below.

<|start_header_id|>user<|end_header_id|>

Below are two audits for a piece of solidity code. One is your audit which you can assume is always correct.
You are a security auditor and your task is to mark an audit written by a junior auditor based on 3 critera.

First reason about wether each criterion is met or not, without outputing wether the criterion is satisfied or not.
Only after you have explained the reasoning behind each criterion, output a list labeling each criterion with either "PASS" or "FAIL".

Reason about wether the junior auditor's description satisfies the following criteria:
Criterion 1: Fulfilled if the two audits describe the same vulnerability. This is about the nature of vulnerability, rather than the specifics of the code.
Criterion 2: Fulfilled if the junior auditor describes the same code scope or a subset of yours. In other words partial scope is also acceptable. Otherwise, this criterion is not met.
Criterion 3: Fulfilled if the two audits describe the same attacking strategy, even if the junior audit's strategy lacks the same level of detail as yours.

Right after this output EXACTLY 3 criteria with a status of either "PASS" or "FAIL".
Example: Criterion 1: PASS

If criteria are not fully failed or satisifed, determine either "PASS" or "FAIL" is more appropriate label.
Still the labels need to be either "PASS" or "FAIL" and nothing else is allowed.
Do not output anything else like clarifications or other remarks after the list of criteria.

Your audit:
{}<|eot_id|>

Junior Auditor's audit:
{}<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
Reasoning about criteria:
"""

In [341]:
comparison_query_template = \
"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Below is text comprehension task. Output a score based on the specification below.

<|start_header_id|>user<|end_header_id|>

Below are two audits for a piece of solidity code.
You are a security auditor and your task is to mark an audit written by a student based on 3 critera.

First reason about whether each criterion is met or not.
If criteria are not fully satisifed, determine which is the more appropriate label from "PASS" or "FAIL" (nothing else is allowed).
Only after you have explained the reasoning behind each criterion, output a list labeling each criterion with either "PASS" or "FAIL".

Reason about wether the student's description satisfies the following criteria:
Criterion 1: Fulfilled if the two audits describe an identical vulnerability. This is about the nature of vulnerability and how it affects the contract, rather than the specifics of the code.
Criterion 2: Fulfilled if the student describes the same code scope or a subset of yours. In other words partial scope is also acceptable. Otherwise, this criterion is not met.
Criterion 3: Fulfilled if the two audits describe identical attacking strategy. The description does NOT need to be identical just the idea for the attack.

Only once you have deliberated on the criteria, output EXACTLY 3 criteria with a status of either "PASS" or "FAIL".
Example: Criterion 1: PASS

Do not output anything else like clarifications or other remarks after the list of criteria.

Your audit:
{}<|eot_id|>

Student's audit:
{}<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
My reasoning about the criteria:
"""

In [358]:
for idx, result in enumerate(run_query(test_data, comparison_query_template_fix, show_tqdm=False)):
    entry = test_data.iloc[idx]

    verdict = result[:-1]
    correct = list(entry[['cr0', 'cr1', 'cr2', 'cr3']])

    if verdict == correct:
        print(f"Test {idx+1}/{len(test_data)}: passed!")
    else:
        print(f"Test {idx+1}/{len(test_data)}: failed!\n")

        print(f"Verdict: {verdict}")
        print(f"Correct: {correct}")
        print("=" * 200)
        print(entry['real'].replace('\\n', '\n'))
        print("-" * 200)
        print(entry['output'].replace('\\n', '\n'))
        print("-" * 200)
        print(result[-1].replace('\\n', '\n'))
        print("=" * 200)
        print()

Test 1/13: passed!
Test 2/13: passed!
Test 3/13: passed!
Test 4/13: passed!
Test 5/13: passed!
Test 6/13: passed!
Test 7/13: passed!
Test 8/13: passed!
Test 9/13: passed!
Test 10/13: passed!
Test 11/13: passed!
Test 12/13: passed!
Test 13/13: passed!


In [ ]:
# 7, 9 (partial again), 10